In [109]:
import pandas as pd
import numpy as np

application = pd.read_csv('/Users/will/Documents/GitHub/consumer-credit-risk/coursework2_application_sampled.csv')
performance = pd.read_csv('/Users/will/Documents/GitHub/consumer-credit-risk/coursework2_performance_data_sampled.csv')

# merge the data from the two csvs
data_full = performance.merge(application, on='LOAN SEQUENCE NUMBER', how='left')

# if column value is RA then it is 0
data_full['CURRENT LOAN DELINQUENCY STATUS'] = data_full['CURRENT LOAN DELINQUENCY STATUS'].apply(lambda x: 0 if x == 'RA' else x)

# change to int
data_full['CURRENT LOAN DELINQUENCY STATUS'] = data_full['CURRENT LOAN DELINQUENCY STATUS'].astype(int)
# change the column CURRENT LOAN DELINQUENCY STATUS
# slide 13 4 intro to analytics
# if value is less then 3 then it is 0 # else it is 1 # zero represents no default # one represents default
data_full['DEFAULT'] = data_full['CURRENT LOAN DELINQUENCY STATUS'].apply(lambda x: 0 if x < 3 else 1)

# add boolean for whether loan in delinquency or not 
data_full['DELINQUENCY'] = data_full['CURRENT LOAN DELINQUENCY STATUS'].apply(lambda x : 1 if x != 0 else 0)

# change monthly reoprting period to datetime
data_full['MONTHLY REPORTING PERIOD'] = pd.to_datetime(data_full['MONTHLY REPORTING PERIOD'], format='%Y%m')

# make a function that takes in a pd.series of zeroes and ones and then returns a new series with every 12 before the ones being ones as well
def default_next_12(default_vector):
     
    if np.where(default_vector==1)[0].size==0:
        return default_vector, default_vector
     
    first_default = np.where(default_vector==1)[0][0]

    remove_series = np.zeros(len(default_vector))

    remove_series[first_default:] = 1

    default_next_12_series = remove_series

    remove_series[first_default-12:] = 1

    return(default_next_12_series, remove_series)

default_next_12_col = []
remove_col = []

# group the data by LOAN SEQUENCE NUMBER
for _, group in data_full.groupby('LOAN SEQUENCE NUMBER'):
    default_next_12_series, remove_series = default_next_12(group['CURRENT LOAN DELINQUENCY STATUS'])

    default_next_12_col.extend(default_next_12_series)
    remove_col.extend(remove_series)

data_full['DEFAULT_NEXT_12'] = [x == 1 for x in default_next_12_col]
data_full[[x==0 for x in remove_col]]

data_full_train = data_full[data_full['MONTHLY REPORTING PERIOD'] < '2022-06-01']

# make test data
data_full_test = data_full[data_full['MONTHLY REPORTING PERIOD'] == '2023-06-01']


/var/folders/q7/dvzw6sn17079c5z05_t3tqpc0000gn/T/ipykernel_47803/4139976727.py:4: DtypeWarning: Columns (26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  application = pd.read_csv('/Users/will/Documents/GitHub/consumer-credit-risk/coursework2_application_sampled.csv')
/var/folders/q7/dvzw6sn17079c5z05_t3tqpc0000gn/T/ipykernel_47803/4139976727.py:5: DtypeWarning: Columns (3,7,23,24,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  performance = pd.read_csv('/Users/will/Documents/GitHub/consumer-credit-risk/coursework2_performance_data_sampled.csv')


In [110]:
data_full.head()

,LOAN SEQUENCE NUMBER,MONTHLY REPORTING PERIOD,CURRENT ACTUAL UPB,CURRENT LOAN DELINQUENCY STATUS,LOAN AGE,REMAINING MONTHS TO LEGAL MATURITY,DEFECT SETTLEMENT DATE,MODIFICATION FLAG,ZERO BALANCE CODE,ZERO BALANCE EFFECTIVE DATE,...,SUPER CONFORMING FLAG,PRE-RELIEF REFINANCE LOAN SEQUENCE NUMBER,PROGRAM INDICATOR,RELIEF REFINANCE INDICATOR,PROPERTY VALUATION METHOD,INTEREST ONLY INDICATOR (I/O INDICATOR),MI CANCELLATION INDICATOR,DEFAULT,DELINQUENCY,DEFAULT_NEXT_12
0,F14Q10000351,2021-01-01,194781.21,0,83,277,NaN,NaN,NaN,NaN,...,NaN,NaN,9,NaN,9,N,9,0,0,False
1,F14Q10000351,2021-02-01,194383.73,0,84,276,NaN,NaN,NaN,NaN,...,NaN,NaN,9,NaN,9,N,9,0,0,False
2,F14Q10000351,2021-03-01,193984.68,0,85,275,NaN,NaN,NaN,NaN,...,NaN,NaN,9,NaN,9,N,9,0,0,False
3,F14Q10000351,2021-04-01,193584.05,0,86,274,NaN,NaN,NaN,NaN,...,NaN,NaN,9,NaN,9,N,9,0,0,False
4,F14Q10000351,2021-05-01,193181.83,0,87,273,NaN,NaN,NaN,NaN,...,NaN,NaN,9,NaN,9,N,9,0,0,False


In [ ]:
# cum sum of the current loan delinquency status
data_full.groupby('LOAN SEQUENCE NUMBER')['DELINQUENCY'].cumsum()

# last month current loan deliquency status
data_full.groupby('LOAN SEQUENCE NUMBER')['CURRENT LOAN DELINQUENCY STATUS'].shift(1)

# rolling 12 months time in deliquency
data_full.groupby('LOAN SEQUENCE NUMBER')['DELINQUENCY'].rolling(12, min_periods=1).sum()

LOAN SEQUENCE NUMBER         
F14Q10000351          0         NaN
                      1         NaN
                      2         NaN
                      3         NaN
                      4         NaN
                                 ..
F24Q10213872          1759935   NaN
F24Q10214372          1759936   NaN
                      1759937   NaN
F24Q10214745          1759938   NaN
                      1759939   NaN
Name: DELINQUENCY, Length: 1759940, dtype: float64

LOAN SEQUENCE NUMBER    
F14Q10000351          0     0.0
                      1     0.0
                      2     0.0
                      3     0.0
                      4     0.0
                      5     0.0
                      6     0.0
                      7     0.0
                      8     0.0
                      9     0.0
                      10    0.0
                      11    0.0
                      12    0.0
                      13    0.0
                      14    0.0
                      15    0.0
                      16    0.0
                      17    0.0
                      18    0.0
                      19    0.0
Name: DELINQUENCY, dtype: float64